# Replication Michael P. Keane & Kenneth I. Wopin

This notebook replicates results from: 

Keane, M. P. & Wopin, W. I. (1997). [The career Decisions of Young Men.](https://www.jstor.org/stable/10.1086/262080)
 *Journal of Political Economy, 105(3): 473-552.*

The above mentioned paper builds from a theoratical paper from:

Keane, M. P. & Wopin, W. I., & Vytlacil, E. J. (2011). 
[The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence.](https://www.jstor.org/stable/2109768)
 *The Review of Economics and Statistics, 76(4): 648-672.*

For this purpose l rely on *respy*. Respy is an open-source Python package for the simulation and estimation of a prototypical finite-horizon discrete choice dynamic programming model.


# Groundwork 

This project builds upon collaboration and contributions to the *respy* and *estimagic* open source pacakges. In particular, l have worked with some of the open source contributors of these packages to develop parts of the code.

For the *respy* package, l have integrated the results of a project l previously completed on [benchmarking various implementations of ols and iv ](https://github.com/lindamaok899/least_squares). This was done to drop all statsmodels dependencies in the package in favor of incorporating a faster method of calculating ols estimates. I have also integrated a root mean squared error (rmse) function in the package for the same purpose.

For the *estimagic* package, l have worked on the documentation of the numerical optimisers used in the package. In particular, l have created reference dictionaries for the different arguments that relate to the various implemented numerical optimizers.

For both packages and an additional project (*uncertainty quantification project*), l have also calculated standard errors for parameter estimates of likelihood models. More specifically, l write functions that construct the full covariance matrix of the estimated parameters. There are different ways to calculate those covariance matrices, but in all cases they are based on derivatives of the log likelihood function. I have also written a function that extracts the diagonal elements of these covariance matrices to contstuct a 1d array of containing the standard errors. After this, l have written a benchmarking module that compares the functions to the statsmodels output to ensure complete accuracy of the covariance matrices. Lastly, l have written tests to all the encorprated functions. The under-the-hood workings of this part of my assignment can be viewed in an independent repository: [standard_errors](https://github.com/lindamaok899/standard_errors). Subsequently, l will encorporate the neccesary functions for the replication in the auxilliary folder.
